# HR-CAM for multitask model
This file contains the HR-CAM model and code to get the HR-CAMs for all the 4 tasks of multitask.

PS. Highly recommended to read HR-CAMs papaer if not yet done


In [ ]:
# importing as usual
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow import keras

from sklearn.preprocessing import LabelEncoder


from tensorflow.keras import backend as K
import nibabel as nib
import cv2
import time
from skimage.transform import resize
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from tensorflow.keras.utils import to_categorical

from prep_data import get_roi, get_all_subjects, get_subject, get_subject_list, pad_to_shape, remove_padding
from PatchGenerator import PatchGenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping, TensorBoard
from tensorflow.keras.applications.resnet50 import preprocess_input
import models

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping, TensorBoard
# import models
from tensorflow.keras import Model
from tensorflow.keras.applications import ResNet50,DenseNet169,InceptionResNetV2,VGG16
from tensorflow.keras.layers import Conv2DTranspose

from tensorflow.keras.layers import Add, MaxPool2D, Flatten,concatenate,Input,Activation
from tensorflow.keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization, Lambda, AveragePooling2D

In [ ]:
# Importing data

x_train = np.load('../Data/X_train_ML_3.npy')
y_train = np.load('../Data/Y_train_ML_3.npy')
x_test = np.load('../Data/X_test_ML_3.npy')
y_test = np.load('../Data/Y_test_ML_3.npy')


print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

In [ ]:
bkp_X_train = np.copy(x_train)
bkp_Y_train = np.copy(y_train)
bkp_X_test = np.copy(x_test)
bkp_Y_test = np.copy(y_test)

In [ ]:
def grayscale_to_3channel(x):
    x = np.squeeze(x)
    return np.stack([x,x,x],axis=-1)

X_train = grayscale_to_3channel(bkp_X_train)
X_test = grayscale_to_3channel(bkp_X_test)

input_shape = X_train.shape[1:]
print(input_shape)


In [ ]:
del(bkp_X_test)
del(bkp_X_train)
del(bkp_Y_test)
del(bkp_Y_train)

In [ ]:
# Importing 3 modality stacked data
x_train = np.load('../Data/X_train_ML_3mod.npy')
y_train = np.load('../Data/Y_train_ML_3mod.npy')
x_test = np.load('../Data/X_test_ML_3mod.npy')
y_test = np.load('../Data/Y_test_ML_3mod.npy')


In [ ]:
datagen = ImageDataGenerator(
    width_shift_range = [0.2, 0.3],
    height_shift_range = [0.2, 0.3],    
    vertical_flip = True,
    horizontal_flip = True,
    rotation_range=15,
    zoom_range=[0.5,1.0],
    #channel_shift_range=0.2
#     fill_mode = 'constant',
#     cval=0,
)

In [ ]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(4)])

In [ ]:
batch_size = 8

train_datagen = datagen.flow(X_train,y_train,
                             batch_size=batch_size,
                             shuffle=True,                             
                            )


test_datagen = ImageDataGenerator().flow(X_test,y_test,batch_size=batch_size,
                                         shuffle=True)

#test_datagen = datagen.flow(X_test,y_test,batch_size=batch_size,
#                                         shuffle=True)

print(len(train_datagen),len(test_datagen))

new_model.save('../Model/TCGA_miccai_trained_2.h5')

new_model.save_weights('weights/TCGA_miccai_trained_2.hdf5')

In [ ]:
# importing the trained Multitask-model
new_model = keras.models.load_model('../Model/TCGA_miccai_trained_2.h5')

In [ ]:
#loading file weights just to be sure
filepath_init = 'weights/TCGA_miccai_trained_2.hdf5'
new_model.load_weights(filepath_init)

In [ ]:
# Adjusting trainability of layers 
# Making all the layers non-trainable
for layer in new_model.layers:
    layer.trainable=False

In [ ]:
new_model.summary()

In [ ]:
def new_model_hr(ip):
    """
    This code defines the HR-model with 4 layers from each task's branch
    
    """
    model_hr = new_model
    
    for layer in new_model.layers:
        layer.trainable=False
    for layer in model_hr.layers:
        layer.trainable=False
        
    imp_layer1_ids = ['fireop1_0/squeeze1x1','fireop1_0/squeeze1x1','fireop1_0/squeeze1x1','fireop1_0/expand3x3']
    out1_1 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer1_ids[0]).output)             
    out1_2 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer1_ids[1]).output)             
    out1_3 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer1_ids[2]).output)            
    out1_4 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer1_ids[3]).output)
    
    merge1 = concatenate([out1_1,out1_2,out1_3,out1_4],axis=-1, name='merge1')               
    op1 = Dense(1,activation='sigmoid',name='dense_hr_1')(merge1)    


    imp_layer2_ids = ['fireop2_0/expand3x3','fireop2_0/expand1x1','fireop2_0/squeeze1x1','fireop2_0/squeeze1x1'] 
    out2_1 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer2_ids[0]).output)             
    out2_2 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer2_ids[1]).output)             
    out2_3 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer2_ids[2]).output)            
    out2_4 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer2_ids[3]).output)
    
    merge2 = concatenate([out2_1,out2_2,out2_3,out2_4],axis=-1, name='merge2')               
    op2 = Dense(1,activation='sigmoid',name='dense_hr_2')(merge2)   

    imp_layer3_ids = ['fireop3/expand3x3','fireop3/expand1x1','fireop3/squeeze1x1','fireop3_1/expand3x3'] 
    out3_1 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer3_ids[0]).output)             
    out3_2 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer3_ids[1]).output)             
    out3_3 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer3_ids[2]).output)            
    out3_4 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer3_ids[3]).output)
    
    merge3 = concatenate([out3_1,out3_2,out3_3,out3_4],axis=-1, name='merge3')              
    op3 = Dense(1,activation='sigmoid',name='dense_hr_3')(merge3)   

    imp_layer4_ids = ['fireop4_0/expand3x3','fireop4_0/expand1x1','fireop4_0/squeeze1x1','fireop4_1/expand3x3'] 
    out4_1 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer4_ids[0]).output)             
    out4_2 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer4_ids[1]).output)             
    out4_3 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer4_ids[2]).output)            
    out4_4 = GlobalAveragePooling2D()(model_hr.get_layer(imp_layer4_ids[3]).output)
    
    merge4 = concatenate([out4_1,out4_2,out4_3,out4_4],axis=-1, name='merge4')               
    op4 = Dense(1,activation='sigmoid',name='dense_hr_4')(merge4)    

    
    return [op1, op2, op3, op4]
    

In [ ]:
#ip = np.zeros((1,128,128,3)).astype('float32')
hr_model=  Model(inputs = new_model.inputs, outputs = new_model_hr(new_model.inputs))

In [ ]:
hr_model.summary()

In [ ]:
hr_filename=os.path.join('logs','TCGA_hr.csv')
hr_filepath=os.path.join('weights','TCGA_hr.hdf5')
hr_csv_log = CSVLogger(hr_filename, separator=',', append=True)
hr_checkpoint = ModelCheckpoint(hr_filepath, monitor='val_dense_hr_1_acc', verbose=1, save_best_only=True)
hr_rl = ReduceLROnPlateau(monitor='val_loss',patience=10,min_delta=0.001,cooldown=5,)
hr_tb = TensorBoard('./logs',histogram_freq=0)
hr_callbacks_list = [hr_csv_log,
                  hr_checkpoint,
                  #hr_rl,
                  hr_tb,
                 ]

In [ ]:
hr_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0003),loss=['binary_crossentropy','binary_crossentropy','binary_crossentropy','binary_crossentropy' ], metrics=['accuracy'])

In [ ]:
# Loading weights agaiin from previous model 
hr_model.load_weights(filepath_init, by_name=True)

In [ ]:
train_steps = len(train_datagen)
test_steps = len(test_datagen)
print(train_steps, test_steps)

In [ ]:
# Training the HR model
epochs = 20
HR_H = hr_model.fit(generator_wrapper(train_datagen), steps_per_epoch=train_steps, epochs=epochs,
                    verbose = 1,validation_data=generator_wrapper(test_datagen),validation_steps=test_steps,callbacks=hr_callbacks_list)

In [ ]:
hr_model.load_weights(hr_filepath)

In [ ]:
_test_data, _test_labels = test_datagen.__getitem__(np.random.randint(0,len(test_datagen)))

In [ ]:
## evaluate the model and predict on testing data
print('Evaluate')
a=hr_model.evaluate(generator_wrapper(test_datagen),batch_size=batch_size,verbose=1,steps = 1)
print('val_loss , val_acc: ', a)

In [ ]:
def get_heatmaps_hr(model, image, layer_ids, weight_layer=None, classes=None, h=None, w=None):
    #Function to compute the final hr_cams from the trained hr_cam model
    #model : Trained hr_cam model
    #image : images to compute hr_cams for
    #layer_ids : layer ids of important layers used in the hr_cams
    #weight_layer : layer of the model from which the weights need to be computed set to -1th layer by default
    #classes : number of classes 
    #h = height of image, should be specified if output height needs to be different than the input image
    #w = width of image, should be specified if output width needs to be different than the input image
    
    if weight_layer is None:
        weight_layer = model.get_layer(index=-1)
    heatmaps = []
    wts = []
    
    h = image.shape[-3] if h == None else h
    w = image.shape[-2] if w == None else w
    
    if classes is None:
        pred = model.predict(image)
        classes = pred[0]
    
    imp_layer_0 = model.get_layer(layer_ids[0])
    imp_layer_1 = model.get_layer(layer_ids[1])
    imp_layer_2 = model.get_layer(layer_ids[2])
    imp_layer_3 = model.get_layer(layer_ids[3])
   
    hmp_model = Model(inputs=model.inputs, outputs=[imp_layer_0.output,
                                                    imp_layer_1.output,
                                                    imp_layer_2.output,
                                                    imp_layer_3.output
                                                    ])
    
    out = hmp_model.predict(x=image,verbose=1,batch_size=int(batch_size/4))
    weights = model.get_layer(weight_layer).get_weights()[0] # [0] is for wts [1] for bias
 
    for i in range(len(classes)):
        
        print('Image no : ',i)
                
        class_idx = classes[i]           
        class_w = weights

        i_idx = 0
        f_idx = imp_layer_0.output_shape[-1]
        hmp_0 = K.sum(out[0][i:i+1] * class_w[i_idx:f_idx,0], axis=-1)
        hmp_0 = K.expand_dims(hmp_0, axis=-1)
        hmp_0 = K.eval(tf.image.resize_bilinear(hmp_0, [h, w]))

        i_idx = imp_layer_0.output_shape[-1]
        f_idx = i_idx + imp_layer_1.output_shape[-1]
        hmp_1 = K.sum(out[1][i:i+1] * class_w[i_idx:f_idx,0], axis=-1)
        hmp_1 = K.expand_dims(hmp_1, axis=-1)
        hmp_1 = K.eval(tf.image.resize_bilinear(hmp_1, [h, w]))

        i_idx = imp_layer_1.output_shape[-1]
        f_idx = i_idx + imp_layer_2.output_shape[-1]
        hmp_2 = K.sum(out[2][i:i+1] * class_w[i_idx:f_idx,0], axis=-1)
        hmp_2 = K.expand_dims(hmp_2, axis=-1)
        hmp_2 = K.eval(tf.image.resize_bilinear(hmp_2, [h, w]))

        i_idx = imp_layer_2.output_shape[-1]
        f_idx = i_idx + imp_layer_3.output_shape[-1]
        hmp_3 = K.sum(out[3][i:i+1] * class_w[i_idx:f_idx,0], axis=-1)
        hmp_3 = K.expand_dims(hmp_3, axis=-1)
        hmp_3 = K.eval(tf.image.resize_bilinear(hmp_3, [h, w]))

        
        hmp = np.squeeze(hmp_0+hmp_1+hmp_2+hmp_3)
        heatmaps.append(hmp)
        wts.append(class_w)    
    
    return np.stack(heatmaps,axis = 0), classes, np.stack(class_w,axis=0)

In [ ]:
imp_layer1_ids = ['fireop1_0/squeeze1x1','fireop1_0/squeeze1x1','fireop1_0/squeeze1x1','fireop1_0/expand3x3']
imp_layer2_ids = ['fireop2_0/expand3x3','fireop2_0/expand1x1','fireop2_0/squeeze1x1','fireop2_0/squeeze1x1']
imp_layer3_ids = ['fireop3/expand3x3','fireop3/expand1x1','fireop3/squeeze1x1','fireop3_1/expand3x3'] 
imp_layer4_ids = ['fireop4_0/expand3x3','fireop4_0/expand1x1','fireop4_0/squeeze1x1','fireop4_1/expand3x3']
output_layer_ids = ['dense_hr_1', 'dense_hr_2', 'dense_hr_3', 'dense_hr_2']

In [ ]:
# selecting images to get the Cams for
_x = np.copy(X_test)[[2, 54, 28, 98, 140, 20, 89],:]
_y = np.copy(y_test)[[2, 54, 28, 98, 140, 20, 89]]

In [ ]:
print('Calculating HR hmp')
hr_hmp_0, hr_pred_0, hr_w1_0 = get_heatmaps_hr(hr_model,_x, layer_ids = imp_layer1_ids, weight_layer=output_layer_ids[0] )


In [ ]:
inv_hr_hmp_0, inv_hr_pred_0, inv_hr_w1_0 =get_heatmaps_hr(hr_model,_x, layer_ids = imp_layer1_ids, weight_layer=output_layer_ids[0] )


In [ ]:
for i in range(len(hr_hmp_0)):
    hr_hmp_0[i] = np.maximum(hr_hmp_0[i],0,hr_hmp_0[i])
    hr_hmp_0[i] = (hr_hmp_0[i]-hr_hmp_0[i].min())/(hr_hmp_0[i].max()-hr_hmp_0[i].min())
for i in range(len(inv_hr_hmp_0)):
    hr_hmp_0[i] = np.maximum(hr_hmp_0[i],0,hr_hmp_0[i])
    inv_hr_hmp_0[i] = (inv_hr_hmp_0[i]-inv_hr_hmp_0[i].min())/(inv_hr_hmp_0[i].max()-inv_hr_hmp_0[i].min())

In [ ]:
#GRADE HR_CAMS

idx = 6#np.random.randint(0,len(_y))
x_img = np.copy(_x[idx])
y_img = np.copy(_y[idx])
print('Idx: ',idx,' Class: ',y_img)
#print('Predicted class - Base model: ',res_pred[idx],' HR_Cam: ',hr_pred[idx])

plt.figure(figsize=(20, 20))

plt.subplot(1, 3, 1)
img = cv2.resize(np.squeeze(x_img[:,:,0]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')

plt.subplot(1, 3, 2)
output_0 = cv2.resize(np.squeeze(inv_hr_hmp_0[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_0, 'jet', interpolation='none', alpha=0.50)

"""
plt.subplot(1, 4, 3)
output_0 = cv2.resize(np.squeeze(grad_hmp[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_0, 'jet', interpolation='none', alpha=0.50)
"""
plt.subplot(1, 3, 3)
output_0 = cv2.resize(np.squeeze(hr_hmp_0[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_0, 'jet', interpolation='none', alpha=0.50)

plt.show()

In [ ]:
print('Calculating HR hmp')
hr_hmp_1, hr_pred_1, hr_w1_1 = get_heatmaps_hr(hr_model,_x, layer_ids = imp_layer2_ids, weight_layer=output_layer_ids[1] )


In [ ]:
inv_hr_hmp_1, inv_hr_pred_1, inv_hr_w1_1 =get_heatmaps_hr(hr_model,_x, layer_ids = imp_layer2_ids,
                                                    weight_layer=output_layer_ids[1] )


In [ ]:
for i in range(len(hr_hmp_1)):
    hr_hmp_1[i] = np.maximum(hr_hmp_1[i],0,hr_hmp_1[i])
    hr_hmp_1[i] = (hr_hmp_1[i]-hr_hmp_1[i].min())/(hr_hmp_1[i].max()-hr_hmp_1[i].min())
for i in range(len(inv_hr_hmp_1)):
    hr_hmp_1[i] = np.maximum(hr_hmp_1[i],0,hr_hmp_1[i])
    inv_hr_hmp_1[i] = (inv_hr_hmp_1[i]-inv_hr_hmp_1[i].min())/(inv_hr_hmp_1[i].max()-inv_hr_hmp_1[i].min())

In [ ]:
#IDH HR_CAMS

#idx = np.random.randint(0,len(_y))
x_img = np.copy(_x[idx])
y_img = np.copy(_y[idx])
print('Idx: ',idx,' Class: ',y_img)
#print('Predicted class - Base model: ',res_pred[idx],' HR_Cam: ',hr_pred[idx])

plt.figure(figsize=(20, 20))

plt.subplot(1, 3, 1)
img = cv2.resize(np.squeeze(x_img[:,:,0]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')

plt.subplot(1, 3, 2)
output_1 = cv2.resize(np.squeeze(inv_hr_hmp_1[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(-output_1, 'jet', interpolation='none', alpha=0.50)
"""
plt.subplot(1, 4, 3)
output_0 = cv2.resize(np.squeeze(grad_hmp[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_0, 'jet', interpolation='none', alpha=0.50)
"""
plt.subplot(1, 3, 3)
output_0 = cv2.resize(np.squeeze(hr_hmp_1[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_0, 'jet', interpolation='none', alpha=0.50)

plt.show()

In [ ]:
print('Calculating HR hmp')
hr_hmp_2, hr_pred_2, hr_w1_2 = get_heatmaps_hr(hr_model,_x, layer_ids = imp_layer3_ids, 
                                         weight_layer=output_layer_ids[2] )

In [ ]:
inv_hr_hmp_2, inv_hr_pred_2, inv_hr_w1_2 =get_heatmaps_hr(hr_model,_x, layer_ids = imp_layer3_ids,
                                                    weight_layer=output_layer_ids[2] )


In [ ]:
for i in range(len(hr_hmp_2)):
    hr_hmp_2[i] = np.maximum(hr_hmp_2[i],0,hr_hmp_2[i])
    hr_hmp_2[i] = (hr_hmp_2[i]-hr_hmp_2[i].min())/(hr_hmp_2[i].max()-hr_hmp_2[i].min())
for i in range(len(inv_hr_hmp_2)):
    hr_hmp_2[i] = np.maximum(hr_hmp_2[i],0,hr_hmp_2[i])
    inv_hr_hmp_2[i] = (inv_hr_hmp_2[i]-inv_hr_hmp_2[i].min())/(inv_hr_hmp_2[i].max()-inv_hr_hmp_2[i].min())

In [ ]:
#MGMT HR_CAMS

#idx = np.random.randint(0,len(_y))
x_img = np.copy(_x[idx])
y_img = np.copy(_y[idx])
print('Idx: ',idx,' Class: ',y_img)
#print('Predicted class - Base model: ',res_pred[idx],' HR_Cam: ',hr_pred[idx])

plt.figure(figsize=(20, 20))

plt.subplot(1, 3, 1)
img = cv2.resize(np.squeeze(x_img[:,:,0]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')

plt.subplot(1, 3, 2)
output_2 = cv2.resize(np.squeeze(inv_hr_hmp_2[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_2, 'jet', interpolation='none', alpha=0.50)
"""
plt.subplot(1, 4, 3)
output_0 = cv2.resize(np.squeeze(grad_hmp[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_0, 'jet', interpolation='none', alpha=0.50)
"""
plt.subplot(1, 3, 3)
output_0 = cv2.resize(np.squeeze(hr_hmp_2[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_0, 'jet', interpolation='none', alpha=0.50)

plt.show()

In [ ]:
print('Calculating HR hmp')
hr_hmp_3, hr_pred_3, hr_w1_3 = get_heatmaps_hr(hr_model,_x, layer_ids = imp_layer4_ids, 
                                         weight_layer=output_layer_ids[3] )

In [ ]:
inv_hr_hmp_3, inv_hr_pred_3, inv_hr_w1_3 =get_heatmaps_hr(hr_model,_x, layer_ids = imp_layer4_ids,
                                                    weight_layer=output_layer_ids[3] )


In [ ]:
for i in range(len(hr_hmp_3)):
    hr_hmp_3[i] = np.maximum(hr_hmp_3[i],0,hr_hmp_3[i])
    hr_hmp_3[i] = (hr_hmp_3[i]-hr_hmp_3[i].min())/(hr_hmp_3[i].max()-hr_hmp_3[i].min())
for i in range(len(inv_hr_hmp_3)):
    hr_hmp_3[i] = np.maximum(hr_hmp_3[i],0,hr_hmp_3[i])
    inv_hr_hmp_3[i] = (inv_hr_hmp_3[i]-inv_hr_hmp_3[i].min())/(inv_hr_hmp_3[i].max()-inv_hr_hmp_3[i].min())

In [ ]:
#1p19q HR_CAMS

#idx = np.random.randint(0,len(_y))
x_img = np.copy(_x[idx])
y_img = np.copy(_y[idx])
print('Idx: ',idx,' Class: ',y_img)
#print('Predicted class - Base model: ',res_pred[idx],' HR_Cam: ',hr_pred[idx])

plt.figure(figsize=(20, 20))

plt.subplot(1, 3, 1)
img = cv2.resize(np.squeeze(x_img[:,:,0]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')

plt.subplot(1, 3, 2)
output_0 = cv2.resize(np.squeeze(inv_hr_hmp_3[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(-output_0, 'jet', interpolation='none', alpha=0.50)

"""
plt.subplot(1, 4, 3)
output_0 = cv2.resize(np.squeeze(grad_hmp[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_0, 'jet', interpolation='none', alpha=0.50)
"""
plt.subplot(1, 3, 3)
output_3 = cv2.resize(np.squeeze(inv_hr_hmp_3[idx]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_3, 'jet', interpolation='none', alpha=0.50)

plt.show()

In [ ]:
# plotting all images

plt.figure(figsize=(20, 20))

plt.subplot(1, 5, 1)
img = cv2.resize(np.squeeze(x_img[:,:,0]), tuple(input_shape[0:2]), 0, 0, 0, interpolation=cv2.INTER_LINEAR)
plt.imshow(img, 'gray', interpolation='none')
plt.gca().set_title('Original')
plt.subplot(1, 5, 2)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(output_0, 'jet', interpolation='none', alpha=0.70)
plt.gca().set_title('Grade')
plt.subplot(1, 5, 3)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(-output_1, 'jet', interpolation='none', alpha=0.70)
plt.gca().set_title('IDH')
plt.subplot(1, 5, 4)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(-output_2, 'jet', interpolation='none', alpha=0.70)
plt.gca().set_title('MGMT')
plt.subplot(1, 5, 5)
plt.imshow(img, 'gray', interpolation='none')
plt.imshow(-output_3, 'jet', interpolation='none', alpha=0.70)
plt.gca().set_title('1P/19Q')

plt.show()